<a href="https://colab.research.google.com/github/SriNithin965/cognizant_internship/blob/main/Sme_Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the packages

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from sklearn.metrics import mean_squared_error

Loading the dataset

In [43]:
data = pd.read_csv('https://raw.githubusercontent.com/SriNithin965/cognizant_internship/main/AAPL.csv')

In [ ]:
data.head(10)

In [ ]:
data.dtypes

changing the data type of date column

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])
data.dtypes

Visualization

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(2, 2, 1)
plt.title('Open')
plt.plot(data['Date'],data['Open'])
plt.subplot(2, 2, 2)
plt.title('Close')
plt.plot(data['Date'],data['Close'])
plt.subplot(2, 2, 3)
plt.title('High')
plt.plot(data['Date'],data['High'])
plt.subplot(2, 2, 4)
plt.title('Low')
plt.plot(data['Date'],data['Low'])
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Open')
plt.plot(data['Date'],data['Open'])

In [ ]:
data = data[['Open']]
data.head(10)

Spliting the data train and test

In [50]:
training_size=int(len(data)*0.80)
test_size=len(data)-training_size
train_data,test_data=data[0:training_size],data[training_size:len(data)]

In [ ]:
len(train_data),len(test_data)

Applying the scaling - (MinMaxscaling)

In [52]:
scaler=MinMaxScaler(feature_range=(0,1))
data_trans_train=scaler.fit_transform(np.array(train_data).reshape(-1,1))
data_trans_test=scaler.fit_transform(np.array(test_data).reshape(-1,1))

Preparing the window

In [53]:
def prepare_data(timeseries_data, n_features):
	X, y =[],[]
	for i in range(len(timeseries_data)):
		end_ix = i + n_features
		if end_ix > len(timeseries_data)-1:
			break
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [54]:
window_size = 3
n_features = 1
X_train,y_train = prepare_data(data_trans_train,window_size)
X_test,y_test = prepare_data(data_trans_test,window_size)

Reshaping the data

In [55]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

In [ ]:
print(X_train.shape)
print(X_test.shape)

Model building and training

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(window_size, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
epoch = 50
model.fit(X_train, y_train, epochs=epoch, verbose=1)

Validation on test data

In [ ]:
predict_X=model.predict(X_test)
mean_squared_error(y_test,predict_X)

In [ ]:
plt.plot(y_test)
plt.plot(predict_X)
plt.show()

BYE OR SELL STOCK

In [ ]:
def buyorsell(days):
  window = data_trans_test[len(data_trans_test)-window_size:]
  i=0
  future_stock =[]
  while(i<days):
    window = window.reshape(1,window_size,1)
    value = model.predict(window)
    stock=scaler.inverse_transform(value)
    future_stock.append(stock[0][0])
    window = window.reshape(1,window_size)
    window = window[0,1:]
    window = np.append(window,value)
    i=i+1
  future_stock
  avg = 0
  sum = 0
  print(future_stock)
  for i in range(0,len(future_stock)-1):
    sum = sum + (future_stock[i+1]-future_stock[i])
  avg = sum/(len(future_stock)-1)
  print(avg)
  if(avg<0):
    print('Sell the stocks')
  else:
    print('buy the stocks')

days = int(input('based on how much days of future stocks you wanna decide to buy or sell in range of 1-100:\n'))
buyorsell(days)